This tutorial can be downloaded [link](http://greatfire.uchicago.edu/west-public/West/raw/master/Doc/tutorials/advanced/advanced_001.ipynb).

# Advanced tutorial 1: Symmetry analysis in Quantum Defect Embedding Theory (QDET)

We explain how to identify the symmetry of excited many-body states calculated with the Quantum Defect Embedding Theory (QDET) using the point symmetry of the defect. 
In this tutorial, we focus on the $\mathrm{NV^-}$ center in diamond.

## Step 1: Mean-field starting point

As for a standard QDET calculation, we need to determine the mean-field starting point first. In order to perform the symmetry analysis, we will additionally write the single-particle wavefunctions to cube files, in order to determine their symmetry character.

Download the following files in your working directory:

In [1]:
%%bash
wget -N -q http://www.west-code.org/doc/training/nv_diamond_63/pw.in
wget -N -q http://www.west-code.org/doc/training/nv_diamond_63/pp.in
wget -N -q http://www.quantum-simulation.org/potentials/sg15_oncv/upf/C_ONCV_PBE-1.0.upf
wget -N -q http://www.quantum-simulation.org/potentials/sg15_oncv/upf/N_ONCV_PBE-1.0.upf

We can now inspect the `pw.in` file, the input for the `pw.x` code:

In [2]:
%%bash
cat pw.in

&CONTROL
calculation = 'scf'
wf_collect = .true.
pseudo_dir = './'
/
&SYSTEM
input_dft = 'PBE'
ibrav = 0
ecutwfc = 50
nosym = .true.
tot_charge = -1
nspin = 1
nbnd = 176
occupations = 'from_input'
nat = 63
ntyp = 2
/
&ELECTRONS
conv_thr = 1D-08
/
K_POINTS gamma
CELL_PARAMETERS angstrom
7.136012  0.000000  0.000000
0.000000  7.136012  0.000000
0.000000  0.000000  7.136012
ATOMIC_SPECIES
C  12.01099968  C_ONCV_PBE-1.0.upf
N  14.00699997  N_ONCV_PBE-1.0.upf
ATOMIC_POSITIONS crystal
C    0.99996000  0.99996000  0.99996000
C    0.12495000  0.12495000  0.12495000
C    0.99905000  0.25039000  0.25039000
C    0.12350000  0.37499000  0.37499000
C    0.25039000  0.99905000  0.25039000
C    0.37499000  0.12350000  0.37499000
C    0.25039000  0.25039000  0.99905000
C    0.37499000  0.37499000  0.12350000
C    0.00146000  0.00146000  0.50100000
C    0.12510000  0.12510000  0.62503000
C    0.00102000  0.24944000  0.74960000
C    0.12614000  0.37542000  0.87402000
C    0.24944000  0.00102000  0.74960

We can now run `pw.x` on 2 cores.

In [ ]:
%%bash
mpirun -n 2 pw.x -i pw.in > pw.out

Once the ground state calculation has finished successfully, we can write the single-particle wavefunctions using the `pp.x` executable. The input file is given as

In [3]:
%%bash
cat pp.in

&INPUTPP
  filplot = 'wfct'
  plot_num = 7
  kpoint(1) = 1
  kband(1) = 87
  kband(2) = 128
  lsign = true
  /

&PLOT
  iflag = 3
  output_format = 6
  fileout = '.cube'
/

And the calculation is performed with the following command

In [ ]:
%%bash
mpirun -n 2 pp.x -i pp.in > pp.out

## Step 2: Definition of the point group

We now define the point group. A point group is characterized by two quantities:
* a set of symmetry operations (identity, reflection, and rotation operations)
* a corresponding character table

In [4]:
import numpy as np
from westpy.qdet.symm import PointGroup, PointGroupOperation, PointGroupRotation, PointGroupReflection

sq3 = np.sqrt(3)

# the origin is set to the Nitrogen site.
# The prefactor corresponds to the real-space grid along each direction.
origin = 64 * np.array([0.48731,  0.48731,  0.48731])

point_group = PointGroup(
    name="C3v",
    operations={
        "E": PointGroupOperation(T=np.eye(4)),
        "C3_1": PointGroupRotation(rotvec=2 * np.pi / 3 * np.array([1/sq3, 1/sq3, 1/sq3]),
                                   origin=origin),
        "C3_2": PointGroupRotation(rotvec=4 * np.pi / 3 * np.array([1/sq3, 1/sq3, 1/sq3]),
                                   origin=origin),
        "Cv_1": PointGroupReflection(normal=(1, -1, 0), origin=origin),
        "Cv_2": PointGroupReflection(normal=(0, -1, 1), origin=origin),
        "Cv_3": PointGroupReflection(normal=(-1, 0, 1), origin=origin)
    },
    ctable={
        "A1": [1, 1, 1, 1, 1, 1],
        "A2": [1, 1, 1, -1, -1, -1],
        "E": [2, -1, -1, 0, 0, 0]})

 
 _    _ _____ _____ _____            
| |  | |  ___/  ___|_   _|           
| |  | | |__ \ `--.  | |_ __  _   _  
| |/\| |  __| `--. \ | | '_ \| | | | 
\  /\  / |___/\__/ / | | |_) | |_| | 
 \/  \/\____/\____/  \_/ .__/ \__, | 
                       | |     __/ | 
                       |_|    |___/  
 
WEST version     :  5.2.0
Today            :  2022-12-13 15:23:23.758988


## Step 3: Diagonalization of QDET Hamiltonian

In this tutorial, we focus on the symmetry analysis with QDET. To avoid costly calculations of the matrix elements of the effective Hamiltonian, we simply download the input and output files of the QDET calculation. To learn how to perform QDET calculations, see [Basic Tutorial 5](http://greatfire.uchicago.edu/west-public/West/raw/master/Doc/tutorials/basic/basic_005.ipynb).

In [5]:
%%bash
mkdir -p west.wfreq.save
wget -N -q http://www.west-code.org/doc/training/nv_diamond_63/wfreq.in
wget -N -q http://www.west-code.org/doc/training/nv_diamond_63/wfreq.json -O west.wfreq.save/wfreq.json

To add point-group analysis to QDET, we simply pass the point-group object defined above to the Hamiltonian object. Additionally, we provide the list of filenames for the wavefunctions.

The code will apply the symmetry operations to the Kohn-Sham wavefunctions stored in the cube files and determine the character of these wavefunctions. After the diagonalization of the effective Hamiltonian, the character of the many-body states can then be inferred from the character of the wavefunctions.

In [6]:
# Here, we provide the list of file paths for the cube files for each basis function of the active space.
wfct_filenames = ['wfct_K001_B087.cube','wfct_K001_B122.cube', 'wfct_K001_B123.cube',
                  'wfct_K001_B126.cube', 'wfct_K001_B127.cube', 'wfct_K001_B128.cube']

In [7]:
%%bash
wget -N -q http://www.west-code.org/doc/training/nv_diamond_63/wfct_K001_B087.cube
wget -N -q http://www.west-code.org/doc/training/nv_diamond_63/wfct_K001_B122.cube
wget -N -q http://www.west-code.org/doc/training/nv_diamond_63/wfct_K001_B123.cube
wget -N -q http://www.west-code.org/doc/training/nv_diamond_63/wfct_K001_B126.cube
wget -N -q http://www.west-code.org/doc/training/nv_diamond_63/wfct_K001_B127.cube
wget -N -q http://www.west-code.org/doc/training/nv_diamond_63/wfct_K001_B128.cube

In [8]:
# the diagonalization is exactly the same as without the symmetry analysis.
from westpy.qdet import QDETResult

# construct object for effective Hamiltonian
effective_hamiltonian = QDETResult(filename='west.wfreq.save/wfreq.json', point_group=point_group, wfct_filenames=wfct_filenames)

# diagonalize Hamiltonian
solution = effective_hamiltonian.solve()

PointGroupRep: rep matrices are orthogonal
Irrep of orbitals: ['A1(1.00)', 'A1(1.00)', 'A1(1.00)', 'A1(1.00)', 'E(0.99)', 'E(0.99)']
Solutions are projected onto irreps of C3v group
-----------------------------------------------------
Building effective Hamiltonian...
nspin: 1
occupations: [[2. 2. 2. 2. 1. 1.]]


diag[1RDM - 1RDM(GS)]                                 
  E [eV] char                    87    122    123    126   127   128
0  0.000  3A2                 0.000  0.000  0.000  0.000 0.000 0.000
1  0.436   1E                -0.001 -0.009 -0.018 -0.067 0.037 0.058
2  0.436   1E                -0.001 -0.009 -0.018 -0.067 0.058 0.037
3  1.250  1A1                -0.002 -0.019 -0.023 -0.067 0.056 0.055
4  1.941   3E                -0.003 -0.010 -0.127 -0.860 0.000 1.000
5  1.941   3E                -0.003 -0.010 -0.127 -0.860 1.000 0.000
6  2.937   1E                -0.000 -0.032 -0.043 -0.855 0.110 0.821
7  2.937   1E                -0.000 -0.033 -0.043 -0.855 0.821 0.109
8  4.662  1A1                -0.005 -0.055 -0.188 -1.672 0.960 0.960
9  5.073   3E                -0.014 -0.702 -0.209 -0.075 0.000 1.000

-----------------------------------------------------
